<a href="https://colab.research.google.com/github/simplifiedcomputing/weed_detection/blob/main/weed_detection_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weed Detection Inference
Locate medical plants for human and animal health using object detection and classification.

## Install

- FastAI (training loop library)
- IceVision (computer vision framework)
- MMDetection and Yolo v5 (neural net models)

In [ ]:
!pip install openmim -q
!mim install mmcv-full
!mim install mmdet

!pip install git+git://github.com/airctic/icevision.git#egg=icevision[all] -U -q
!pip install git+git://github.com/airctic/icedata.git -U -q
!pip install yolov5-icevision -U -q 

In [2]:
# Restart kernel after installation
exit()

## Imports


In [3]:
from icevision.all import *
from icevision.models.checkpoint import *

#Loading model
Because we have saved our model weights with the model metadata, we only need to call model_from_checkpoint(checkpoint_path): No other arguments (model_name, backbone_name, classes, img_size) are needed. All the information is already embedded in the checkpoint file.

In [4]:
checkpoint_path = 'https://simplified-computing.de/weed_detection_split_checkpoint.pth'
checkpoint_and_model = model_from_checkpoint(checkpoint_path)

Use load_from_http loader


Downloading: "https://simplified-computing.de/weed_detection_split_checkpoint.pth" to /root/.cache/torch/hub/checkpoints/weed_detection_split_checkpoint.pth


  0%|          | 0.00/27.0M [00:00<?, ?B/s]

In [5]:
# Just logging the info
model_type = checkpoint_and_model["model_type"]
backbone = checkpoint_and_model["backbone"]
class_map = checkpoint_and_model["class_map"]
img_size = checkpoint_and_model["img_size"]
model_type, backbone, class_map, img_size

(<module 'icevision.models.ultralytics.yolov5' from '/usr/local/lib/python3.7/dist-packages/icevision/models/ultralytics/yolov5/__init__.py'>,
 <ClassMap: {'background': 0, 'herbzeitlose': 1}>,
 1920)

In [6]:
model = checkpoint_and_model["model"]

##Transforms

In [7]:
img_size = checkpoint_and_model["img_size"]
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(img_size), tfms.A.Normalize()])

#Data

###Load image

In [ ]:
# cleaning if neccessary
%rm -r image_dir/
%rm -r split_image_dir/

In [ ]:
import requests
url = 'https://simplified-computing.de/weed_detection_DJI_0252.JPG'
r = requests.get(url, allow_redirects=True)
os.mkdir('image_dir')
open('image_dir/image.jpg', 'wb').write(r.content)

#### Extract GPS coordinates

In [10]:
%pip install exif -q
from exif import Image

# load image
img_file = 'image_dir/image.jpg'

# convert to decimal coordinates
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == "S" or ref == "W":
     decimal_degrees = -decimal_degrees
 return decimal_degrees

lon = decimal_coords(Image(img_file).gps_longitude, Image(img_file).gps_longitude_ref)
lat = decimal_coords(Image(img_file).gps_latitude, Image(img_file).gps_latitude_ref)
lon, lat

# image dimensions
#Image(img_file).pixel_x_dimension, Image(img_file).pixel_y_dimension

     |████████████████████████████████| 64 kB 2.4 MB/s 


(10.851753781876983, 49.858426539075396)

#### Split images in smaller pieces

In [11]:
import imageio
path = 'image_dir'

def make_splits(path):
    temp = os.listdir(path)
    img_names = []
    for img in temp:
        img_names.append(path + '/' + img)

    temp = img_names[0]
    index = len(temp) - temp[::-1].find('/')
    os.mkdir('split_image_dir')
    save_path = 'split_image_dir'
    #save_path = temp[:index]
    
    splits_w = [[.0, .5], [.5, 1.]]
    splits_h = [[.0, .5], [.5, 1.]]    
    
    for num, image in enumerate(img_names):
        im = imageio.imread(image)
        h, w = im.shape[:2]
        
        name = image.split('/')[-1]
        for s_h in splits_h:
            for s_w in splits_w:
                img = im[int(h*s_h[0]):int(h*s_h[1]), int(w*s_w[0]):int(w*s_w[1]), :]
                new_w = int(s_w[0]*w)
                new_h = int(s_h[0]*h)
                
                save_name = os.path.join(save_path, f'{num}_{new_h}_{new_w}_split_{name}')
                imageio.imwrite(save_name, img)

make_splits(path)

In [12]:
# list of split files
os.listdir('split_image_dir')

['0_1500_0_split_image.jpg',
 '0_0_0_split_image.jpg',
 '0_1500_2000_split_image.jpg',
 '0_0_2000_split_image.jpg']

In [ ]:
# show sample image
img = PIL.Image.open('split_image_dir/0_0_0_split_image.jpg')
img

# Create dataset

In [14]:
import glob
img_files = glob.glob('split_image_dir/*.jpg')
imgs_array = [PIL.Image.open(Path(name)) for name in img_files]
infer_ds = Dataset.from_images(imgs_array, valid_tfms, class_map=class_map)

#Predict - All at once

Simply call predict:

In [15]:
preds = model_type.predict(model, infer_ds, keep_images=True, detection_threshold=0.01)

#Predict - In batches
If the memory is not enough to predict everything at once, break it down into smaller batches with infer_dataloader:

In [ ]:
infer_dl = model_type.infer_dl(infer_ds, batch_size=1, shuffle=False)
preds = model_type.predict_from_dl(model=model, infer_dl=infer_dl, keep_images=True, detection_threshold=0.01)

  0%|          | 0/4 [00:00<?, ?it/s]

#Visualize

Let´s take a look at the bboxes

In [16]:
preds[0].pred.detection.bboxes

[<BBox (xmin:1489.20849609375, ymin:752.24853515625, xmax:1513.52099609375, ymax:779.3721923828125)>,
 <BBox (xmin:1297.5462646484375, ymin:529.5830078125, xmax:1315.5692138671875, ymax:548.708984375)>,
 <BBox (xmin:1298.7578125, ymin:523.59375, xmax:1325.060302734375, ymax:547.3790283203125)>]

For visualizing the predictions we can use show_preds:

In [ ]:
show_preds(preds=preds)
#show_records(preds)

## Localization

Calculate GPS coordinates for detected objects.

In [ ]:
# 1. grab bboxes
# 2. take mean
# 3. calculate distance to image center
# 4. grab image coords
# 5. show coords for bboxes

## Map

Create Map with detected plants

In [ ]:
%pip install Simplekml
import simplekml
kml = simplekml.Kml()
kml.document.name = "Herbstzeitlose"
kml.newpoint(name="Herbstzeitlose", coords=[(lon,lat)])  # lon, lat, optional height
kml.save('herbstzeitlose.kml')

from google.colab import files
files.download('herbstzeitlose.kml') 